# Testing RF6 Impact Attestation Metrics

Spec [here](https://plaid-cement-e44.notion.site/Impact-Attestations-Data-b8b1c79a624c46ad94305c74def64783)

1. Number of Attestations (only want the count from Citizens and Top Delegates)
2. Number of Attestations by Citizens
3. Number of Attestations by Top delegates
4. Average NPS score of Citizens and Top delegates
5. Most positive superlative
    1. At least 20 citizen or top delegates reviews
    2. Over 95% of those reviews feel “extremely upset”
    3. Over 95% of those reviews rate very likely to recommend (9-10)
6. "Can't live without" superlative
    1. At least 20 citizen or top delegate reviews
    2. Over 90% of those reviews feel “extremely upset”
7. Percentage distribution of different ratings by citizens and top delegates [Extremely upset, Somewhat upset, Neutral]
    1. Percentage distribution of different ratings by citizens
    2. Percentage distribution of different ratings by delegates
8. Number of elected governance members that attested
    1. Number of each group of elected governance members that attested
    2. Average NPS score per elected governance group
    3. Average feeling per elected governance group

In [1]:
import datetime
import json
import numpy as np
import pandas as pd
import random
import secrets

# Load real impact attestations

In [2]:
json_path = 'data/attestations.json'
with open(json_path, 'r') as f:
    data = json.load(f)
print(len(data))    

149


In [3]:
relevant_attestations = []
for a in data:
    qs = a.get('metadata', {}).get('impactAttestations', [])
    
    if len(qs) < 2:
        continue
    
    nps = pmf = None
    for q in qs:
        val = q.get('value')
        if not val:
            continue
        if q['name'] == 'Likely to Recommend':
            nps = val
        elif q['name'] == 'Feeling if didnt exist':
            pmf = val
    
    if not (nps or pmf):
        continue
        
    relevant_attestations.append({
        'id': a['id'],
        'attester': a['attester'],
        'recipient': a['recipient'],
        'farcasterID': a['farcasterID'],
        'projectRegUID': a['projectRegUID'],
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated']),
        'issuer': a['issuer'],
        'metadataurl': a['metadataurl'],
        'nps_score': pd.to_numeric(nps),
        'pmf_score': pd.to_numeric(pmf)
    })

In [4]:
df_attestations_actual = pd.DataFrame(relevant_attestations)
df_attestations_actual.tail(1).T

,15
id,0xecc8d17aa547682ffc8b4b69c1ed59bd642fad4dc6c8...
attester,0x7484aABFef9f39464F332e632047983b67571C0a
recipient,0x5d36a202687fD6Bd0f670545334bF0B4827Cc1E2
farcasterID,"{'type': 'BigNumber', 'hex': '0x2d4c'}"
projectRegUID,0x82169691e4026be1d950f20c2d36500b06fb6f3e0c03...
timeCreated,2024-10-02 06:16:34
issuer,MGL
metadataurl,https://gateway.pinata.cloud/ipfs/QmeYyTJzXsXS...
nps_score,6
pmf_score,2.0


# Create some dummy data attestations

In [5]:
NUM_PROJECTS = 30
NUM_ATTESTERS = 100
NUM_ATTESTATIONS = 1000
PMF_SCALE = 5
NPS_SCALE = 10

COUNCILS = ['Anticapture Commission', 'Code of Conduct Council', 'Collective Feedback Committee',
            'Developer Advisory Board', 'Grants Council', 'Security Council']

projects = {
    f"0x{secrets.token_hex(32)}": random.random()
    for _ in range(NUM_PROJECTS)
}

attesters = {}
for a in range(NUM_ATTESTERS):
    addr = f"0x{secrets.token_hex(20)}"
    
    is_citizen = random.randint(0,10) < 6
    is_delegate = random.randint(0,10) < 3
    councils = set()
    if is_citizen or is_delegate:
        for _ in range(2):
            idx = random.randint(0,10)
            if idx < len(COUNCILS):
                councils.add(COUNCILS[idx])
    attesters.update({
        addr:{
            'is_citizen': is_citizen,
            'is_top_delegate': is_delegate,
            'governance_membership': list(councils)
        }
    })

attestations = []
for i in range(NUM_ATTESTATIONS):
    random_project = random.choice(list(projects.keys()))
    random_attester = random.choice(list(attesters.keys()))
    project_impact = projects.get(random_project)
    pmf_score = random.randint(int(project_impact*PMF_SCALE), PMF_SCALE)
    nps_score = random.randint(int(project_impact*NPS_SCALE), NPS_SCALE)
    attestations.append({
        'uid': f"0x{secrets.token_hex(32)}",
        'attester': random_attester,
        'rf6_applicationUID': random_project,
        'pmf_score': max(1,pmf_score),
        'nps_score': max(1,nps_score),
        **attesters[random_attester]
    })

In [6]:
with open("data/dummy_attestations.json", "w") as f:
    json.dump(attestations,f,indent=2)

attestations[0]

{'uid': '0xa44d12b823e64cc50527bb69beb7d44f7471644e266ee37bcd90f82aaf64d0e4',
 'attester': '0xbbf251e4ba34c74134f332f31e470bee10e5cf90',
 'rf6_applicationUID': '0xac7889f0ea40537723a2120ff73c6d421e21b547029fe0799d16a9e66db5cc7e',
 'pmf_score': 5,
 'nps_score': 7,
 'is_citizen': False,
 'is_top_delegate': False,
 'governance_membership': []}

# Implement a few of the metrics

In [7]:
df_sim = pd.DataFrame(attestations)
df_sim = df_sim[df_sim.is_citizen | df_sim.is_top_delegate]

In [8]:
def most_positive_superlative(rf6_app_uid):
    dff_sim = df_sim[df_sim['rf6_applicationUID'] == rf6_app_uid]
    num_reviews = len(dff_sim)
    pmf_pos_reviews = (dff_sim['pmf_score'] > 4).sum() / num_reviews
    nps_pos_reviews = (dff_sim['nps_score'] > 8).sum() / num_reviews
    
    # lower thresholds for testing only
    result = (num_reviews >= 20) and (pmf_pos_reviews >= .5) and (nps_pos_reviews >= .5)
    
    return str(result)

def cant_live_without_superlative(rf6_app_uid):
    dff_sim = df_sim[df_sim['rf6_applicationUID'] == rf6_app_uid]
    num_reviews = len(dff_sim)
    pmf_pos_reviews = (dff_sim['pmf_score'] > 4).sum() / num_reviews
    
    # lower thresholds for testing only
    result = (num_reviews >= 20) and (pmf_pos_reviews >= .50)
    
    return str(result)

def ratings_distribution(rf6_app_uid):
    dff_sim = df_sim[df_sim['rf6_applicationUID'] == rf6_app_uid]
    dff_pos = dff_sim[dff_sim['pmf_score'] >= 3]
    dff_citizens = dff_sim[dff_sim.is_citizen == True]
    dff_delegates = dff_sim[dff_sim.is_top_delegate == True]
    return {
        'citizens': {
            'extremely_upset': (dff_citizens['pmf_score'] == 5).sum() / len(dff_citizens),
            'somewhat_upset': (dff_citizens['pmf_score'] == 4).sum() / len(dff_citizens),
            'neutral': (dff_citizens['pmf_score'] == 3).sum() / len(dff_citizens)
        },
        'top_delegates': {
            'extremely_upset': (dff_delegates['pmf_score'] == 5).sum() / len(dff_delegates),
            'somewhat_upset': (dff_delegates['pmf_score'] == 4).sum() / len(dff_delegates),
            'neutral': (dff_delegates['pmf_score'] == 3).sum() / len(dff_delegates)
        }
    }

def councils_distribution(rf6_app_uid):
    dff_sim = df_sim[df_sim['rf6_applicationUID'] == rf6_app_uid]
    result = {}
    for council in COUNCILS:
        dff_council = dff_sim[dff_sim['governance_membership'].apply(lambda x: council in x)]
        if dff_council.empty:
            continue
        result.update({
            council.replace(' ','_').lower(): {
                'count_attestations': len(dff_council),
                'avg_pmf_score': dff_council['pmf_score'].mean(),
                'avg_nps_score': dff_council['nps_score'].mean()
            }
        })
    return result

In [9]:
metrics = []
metrics.append({
    'name': 'count_total_attestations',
    'description': 'Number of Attestations (only want the count from Citizens and Top Delegates)',
    'data': df_sim.groupby('rf6_applicationUID')['uid'].count().to_dict()
})
metrics.append({
    'name': 'count_citizen_attestations',
    'description': 'Number of Attestations by Citizens',
    'data': df_sim[df_sim.is_citizen].groupby('rf6_applicationUID')['uid'].count().to_dict()
})
metrics.append({
    'name': 'count_delegate_attestations',
    'description': 'Number of Attestations by Top delegates',
    'data': df_sim[df_sim.is_top_delegate].groupby('rf6_applicationUID')['uid'].count().to_dict()    
})
metrics.append({
    'name': 'avg_nps_score',
    'description': 'Average NPS score of Citizens and Top delegates',
    'data': df_sim.groupby('rf6_applicationUID')['nps_score'].mean().to_dict()
})
metrics.append({
    'name': 'most_positive_superlative',
    'description': 'Most positive superlative (20 reviews, 95% high PMF and 95% high NPS)',
    'data': {uid: most_positive_superlative(uid) for uid in df_sim['rf6_applicationUID'].unique()}
})
metrics.append({
    'name': 'cant_live_without_superlative',
    'description': "Can't live without superlative (20 reviews, 90% high PMF)",
    'data': {uid: cant_live_without_superlative(uid) for uid in df_sim['rf6_applicationUID'].unique()}
})
metrics.append({
    'name': 'percentage_distributions',
    'description': "Percentage distribution of different ratings by citizens and top delegates",
    'data': {uid: ratings_distribution(uid) for uid in df_sim['rf6_applicationUID'].unique()}
})    
metrics.append({
    'name': 'elected_governance_reviews',
    'description': 'Reviews from elected governance members',
    'data': {uid: councils_distribution(uid) for uid in df_sim['rf6_applicationUID'].unique()}
})

In [10]:
with open("data/dummy_metrics.json", "w") as f:
    json.dump(metrics,f,indent=2)